In [2]:
from PIL import Image
import cv2
import numpy as np
import dlib
import matplotlib.pyplot as plt
import os

In [13]:
# Resnet: https://arxiv.org/abs/1512.03385
face_detector = dlib.get_frontal_face_detector()
points_detector = dlib.shape_predictor('C:/Users/tarek/OneDrive/Desktop/Attendance/Weights/shape_predictor_68_face_landmarks.dat')
face_descriptor_extractor = dlib.face_recognition_model_v1('C:/Users/tarek/OneDrive/Desktop/Attendance/Weights/dlib_face_recognition_resnet_model_v1.dat')

In [16]:
mp={}
face_descriptors=None
folder_path='C:/Users/tarek/OneDrive/Desktop/Attendance/cropped/train'

for person in os.listdir(folder_path):
    person_path=folder_path+'/'+person
    for path in os.listdir(person_path):
        img_path=person_path+'/'+path
        mp[img_path]=person



In [17]:
for img_path, person in mp.items():
    # Check if the path is a file before opening it
    if os.path.isfile(img_path):
        image=Image.open(img_path).convert("RGB")
        image_np=np.array(image,'uint8')

        face_detection=face_detector(image_np,1)
        print(person)
        c=0
        for face in face_detection:
            points=points_detector(image_np,face)

            face_descriptor = face_descriptor_extractor.compute_face_descriptor(image_np, points)
            face_descriptor = [f for f in face_descriptor]
            face_descriptor = np.asarray(face_descriptor, dtype=np.float64)
            face_descriptor = face_descriptor[np.newaxis, :]


            if face_descriptors is None:
                face_descriptors=face_descriptor
            else:
                face_descriptors= np.concatenate((face_descriptors, face_descriptor), axis = 0)
    else:
        # If it's a directory, print a message or handle it appropriately
        print(f"Skipping directory: {img_path}")

2020331001
2020331001
2020331001
2020331001
2020331001
2020331001
2020331001


KeyboardInterrupt: 

In [19]:
type(face_descriptors)

numpy.float64

In [20]:

# Path to save the descriptors
save_path = 'C:/Users/tarek/OneDrive/Desktop/Attendance/Model/face_descriptors.npy'

# Save the face_descriptors array
np.save(save_path, face_descriptors)
print(f"Face descriptors saved to {save_path}")


Face descriptors saved to C:/Users/tarek/OneDrive/Desktop/Attendance/Model/face_descriptors.npy


In [21]:
# Load the face_descriptors array
loaded_descriptors = np.load(save_path)
print(f"Loaded face descriptors with shape: {loaded_descriptors.shape}")


Loaded face descriptors with shape: (70, 128)


In [27]:
import pickle

# Save the mp dictionary to a file
def save_mp(mp, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump(mp, f)
    print(f"Dictionary saved to {file_path}")

# Specify the file path
mp_file_path = 'C:/Users/tarek/OneDrive/Desktop/Attendance/Model/mp.pkl'
save_mp(mp, mp_file_path)


Dictionary saved to C:/Users/tarek/OneDrive/Desktop/Attendance/Model/mp.pkl


In [28]:
# Load the mp dictionary from a file
def load_mp(file_path):
    with open(file_path, 'rb') as f:
        loaded_mp = pickle.load(f)
    print(f"Dictionary loaded from {file_path}")
    return loaded_mp

# Load the dictionary
loaded_mp = load_mp(mp_file_path)
print(loaded_mp)


Dictionary loaded from C:/Users/tarek/OneDrive/Desktop/Attendance/Model/mp.pkl
{'C:/Users/tarek/OneDrive/Desktop/Attendance/Dataset10/train_data/Abdullah_Gul/Abdullah_Gul_0001.jpg': 'Abdullah_Gul', 'C:/Users/tarek/OneDrive/Desktop/Attendance/Dataset10/train_data/Abdullah_Gul/Abdullah_Gul_0003.jpg': 'Abdullah_Gul', 'C:/Users/tarek/OneDrive/Desktop/Attendance/Dataset10/train_data/Abdullah_Gul/Abdullah_Gul_0004.jpg': 'Abdullah_Gul', 'C:/Users/tarek/OneDrive/Desktop/Attendance/Dataset10/train_data/Abdullah_Gul/Abdullah_Gul_0005.jpg': 'Abdullah_Gul', 'C:/Users/tarek/OneDrive/Desktop/Attendance/Dataset10/train_data/Abdullah_Gul/Abdullah_Gul_0006.jpg': 'Abdullah_Gul', 'C:/Users/tarek/OneDrive/Desktop/Attendance/Dataset10/train_data/Abdullah_Gul/Abdullah_Gul_0007.jpg': 'Abdullah_Gul', 'C:/Users/tarek/OneDrive/Desktop/Attendance/Dataset10/train_data/Abdullah_Gul/Abdullah_Gul_0008.jpg': 'Abdullah_Gul', 'C:/Users/tarek/OneDrive/Desktop/Attendance/Dataset10/train_data/Abdullah_Gul/Abdullah_Gul_001

In [26]:
type(loaded_descriptors)

numpy.ndarray

In [10]:
np.linalg.norm(face_descriptors[31] - face_descriptors[31])

0.0

In [11]:
mp_list = list(mp.items())

In [12]:
def predict_name(path):
    image=Image.open(path).convert('RGB')
    image_np=np.array(image,'uint8')
    face_detection=face_detector(image_np,1)
    for face in face_detection:
        points=points_detector(image_np,face)

        face_descriptor = face_descriptor_extractor.compute_face_descriptor(image_np, points)
        face_descriptor = [f for f in face_descriptor]
        face_descriptor = np.asarray(face_descriptor, dtype=np.float64)
        face_descriptor = face_descriptor[np.newaxis, :]

        distances = np.linalg.norm(face_descriptor - face_descriptors, axis = 1)
        min_index=np.argmin(distances)
        min_distance=distances[min_index]
        print(min_distance)
        if min_distance<=0.5:
            name_pred=mp_list[min_index][1]
        else:
            name_pred='Undefined'

        #cv2.putText(image_np, 'Pred: ' + str(name_pred), (10, 10), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,0,255))
        #cv2.putText(image_np, 'Exp : ' + str(name_real), (10, 50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,0))
        print(name_pred)

    image_np_rgb = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)

    # Display the image with annotations
    #plt.figure(figsize=(6, 6))
    #plt.imshow(image_np_rgb)
    #plt.axis('off')  # Hide axes
    #plt.title(f"Prediction: {name_pred} | Expected: {name_real}")
    #plt.show()
    return name_pred



In [13]:

test_image_path='C:/Users/tarek/OneDrive/Desktop/Attendance/Dataset10/test_data/Adrien_Brody/Adrien_Brody_0002.jpg'
predict_name(test_image_path)

0.37042756861991755
Adrien_Brody


'Adrien_Brody'

In [14]:
from sklearn.metrics import accuracy_score

true_labels = []
predicted_labels = []

test_path='C:/Users/tarek/OneDrive/Desktop/Attendance/Dataset10/test_data'


In [15]:
for class_idx, class_name in enumerate(os.listdir(test_path)):
    class_dir = os.path.join(test_path, class_name)
    if os.path.isdir(class_dir):
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            true_labels.append(class_name)
            predicted_label = predict_name(img_path)
            predicted_labels.append(predicted_label)
            print(f"True Label: {class_name}, Predicted Label: {predicted_label}")
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

0.26260905854086763
Abdullah_Gul
True Label: Abdullah_Gul, Predicted Label: Abdullah_Gul
0.27672721950998846
Abdullah_Gul
True Label: Abdullah_Gul, Predicted Label: Abdullah_Gul
0.37042756861991755
Adrien_Brody
True Label: Adrien_Brody, Predicted Label: Adrien_Brody
0.30508340202491196
Adrien_Brody
True Label: Adrien_Brody, Predicted Label: Adrien_Brody
0.3828451337541834
Alejandro_Toledo
True Label: Alejandro_Toledo, Predicted Label: Alejandro_Toledo
0.31092263070574816
Alejandro_Toledo
True Label: Alejandro_Toledo, Predicted Label: Alejandro_Toledo
0.4756401379969058
Alvaro_Uribe
True Label: Alvaro_Uribe, Predicted Label: Alvaro_Uribe
0.40881713753803917
Alvaro_Uribe
True Label: Alvaro_Uribe, Predicted Label: Alvaro_Uribe
0.4894663331154778
Amelie_Mauresmo
True Label: Amelie_Mauresmo, Predicted Label: Amelie_Mauresmo
0.4178009716359955
Amelie_Mauresmo
True Label: Amelie_Mauresmo, Predicted Label: Amelie_Mauresmo
0.34861814945346586
Angelina_Jolie
True Label: Angelina_Jolie, Predicted